In [1]:
import serial
import sys
import time

In [2]:
def listSerialPorts():
    """ 
    Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [3]:
listSerialPorts()

['COM3', 'COM4']

In [4]:
port = serial.Serial('COM3', 115200, timeout=0.1)

In [5]:
def readAll(port):
    time.sleep(0.001)
    message = ''
    while port.inWaiting():
        message += port.read(1).decode("utf-8")
    
    return message

In [6]:
readAll(port)

''

In [7]:
def write(port, expression, eol):
    port.flushInput()
    expression = expression.strip()
    expression = expression + eol
    expr_enc = expression.encode()
    port.write(expr_enc)

In [8]:
write(port, 'M114.1', '\n')

In [9]:
readAll(port)

'ok WCS: X:0.0000 Y:0.0000 Z:0.0000\n'

In [12]:
write(port, '?', '\n')
readAll(port)

'ok\n<Idle|MPos:0.0000,0.0000,0.0000,0.0000|WPos:0.0000,0.0000,0.0000|F:4000.0,100.0>\n'

In [9]:
write(port, 'version', '\n')
readAll(port)

'Build version: ~11myvar2, Build date: Jul  8 2021 01:28:43, MCU: LPC1769, System Clock: 120MHz\r\n  CNC Build 4 axis\n'

In [14]:
write(port, 'help', '\n')
readAll(port)

'Commands:\r\nversion\r\nmem [-v]\r\nls [-s] [folder]\r\ncd folder\r\npwd\r\ncat file [limit] [-d 10]\r\nrm file\r\nmv file newfile\r\nremount\r\nplay file [-v]\r\nprogress - shows progress of current play\r\nabort - abort currently playing file\r\nreset - reset smoothie\r\ndfu - enter dfu boot loader\r\nbreak - break into debugger\r\nconfig-set [<configuration_source>] <configuration_setting> <value>\r\nget [pos|wcs|state|status|fk|ik]\r\nget temp [bed|hotend]\r\nset_temp bed|hotend 185\r\nswitch name [value]\r\nnet\r\nload [file] - loads a configuration override file from soecified name or config-override\r\nsave [file] - saves a configuration override file as specified filename or as config-override\r\nupload filename - saves a stream of text to the named file\r\ncalc_thermistor [-s0] T1,R1,T2,R2,T3,R3 - calculate the Steinhart Hart coefficients for a thermistor\r\nthermistors - print out the predefined thermistors\r\nmd5sum file - prints md5 sum of the given file\r\n'

In [23]:
write(port, 'G0 Z50', '\n')

readAll(port)

'ok\n'

In [16]:
readAll(port)

''

In [49]:
write(port, 'M400', '\n')
readAll(port)

''

In [91]:
write(port, 'G0 Z0', '\n')

readAll(port)

'ok\n'

In [62]:
write(port, 'G28 C0', '\n')

readAll(port)

'ok\n'

In [8]:
write(port, 'get pos', '\n')

readAll(port)

'last C: X:0.0000 Y:0.0000 Z:0.0000\nrealtime WCS: X:0.0000 Y:0.0000 Z:0.0000\nMCS: X:0.0000 Y:0.0000 Z:0.0000 A:0.0000\nAPOS: X:0.0000 Y:0.0000 Z:0.0000 A:0.0000\nMP: X:0.0000 Y:0.0000 Z:0.0000 A:0.0000\nCMP: X:0.0000 Y:0.0000 Z:0.0000\n'

In [24]:
write(port, 'M280 S11.2', '\n')

In [29]:
write(port, 'M280 S5.2', '\n')

In [69]:
write(port, 'M281', '\n')

SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

In [ ]:
# pin 2.4 (leftmost small mosfet). Set as a fan control
#M106-on
#M107-off

In [12]:
write(port, 'M106', '\n')

In [13]:
write(port, 'M107', '\n')

In [15]:
write(port, 'M119', '\n')

readAll(port)

'ok\n'

# Testing X

In [12]:
write(port, 'G0 X100 F50000', '\n')

readAll(port)

''

# Testing Y

In [13]:
write(port, 'G0 Y100 F50000', '\n')

readAll(port)

'ok\n'

# Testing Z

In [14]:
write(port, 'G0 Z100 F10000', '\n')

readAll(port)

''

# Testing pipette

In [15]:
write(port, 'G0 A20 F4000', '\n')

readAll(port)

'ok\n'

# Testing endstops one by one

In [11]:
write(port, 'M119', '\n')

readAll(port)

'X_min:1 Y_min:1 Z_min:1 A_min:1 pins- (X)P1.24:1 (Y)P1.26:1 (Z)P1.28:1 (A)P1.29:1 \nok\n'

# Testing homing procedure

In [12]:
write(port, '$H', '\n')

readAll(port)

''

In [11]:
# Moving axes to the middle
write(port, 'G0 X100 Y100 Z100 F10000', '\n')

readAll(port)

write(port, 'G0 A20 F4000', '\n')

readAll(port)

''

# Testing servos

## Magnet servo

In [26]:
# Power servo on
write(port, 'M106', '\n')

In [27]:
# Bottom position
write(port, 'M280 S5.2', '\n')

In [28]:
# Top position
write(port, 'M280 S11.2', '\n')

In [29]:
# Power servo off
write(port, 'M107', '\n')

## Tip drop servo

In [30]:
# Power servo on
write(port, 'M42', '\n')

In [46]:
# drop tip position
write(port, 'M280.1 S7.3', '\n')

In [47]:
# Disengage position
write(port, 'M280.1 S2.5', '\n')

In [48]:
# Power servo off
write(port, 'M43', '\n')

In [24]:
write(port, 'G0 Z0', '\n')

readAll(port)

'ok\n'

In [11]:
write(port, 'G0 X120', '\n')

readAll(port)

'ok\n'

In [14]:
write(port, 'G0 Y90', '\n')

readAll(port)

'ok\n'